In [ ]:
import pandas as pd
import glob
from os.path import basename

path = r'trainData/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    if not basename(filename).startswith('I'):
        continue
    df = pd.read_csv(filename, index_col=None, header=0)
    if df['daytype'].unique()[0] == 0:
        print(filename)
        df.loc[df['daytype'].isin([0]), 'daytype'] = 'Weekday'
        df.loc[df['daytype'].isin([1]), 'daytype'] = 'Weekend'
        
    li.append(df)

In [ ]:
frame = pd.concat(li, axis=0, ignore_index=True, sort=False)

In [ ]:
frame.count()

In [ ]:
frame['daytype'].value_counts()
frame.dropna()

In [ ]:
frame.to_csv('newData/NeueICEs.csv', index=False)

In [ ]:
import pandas as pd
import requests
import json

In [ ]:
df = pd.read_csv('wetterdaten/bahnhof_to_weather_location.csv', index_col=None, header=0)
bundesland = []
land = []
df.head()

In [ ]:
for stadt in df['location']:
    if stadt != 'no-data':
        payload = {'placename': stadt.replace('+', ' '),'maxRows':'1', 'username': 'bwki'}
        #r = requests.get('http://api.geonames.org/postalCodeSearchJSON', params=payload)
        try:
            r = requests.get('http://api.geonames.org/postalCodeSearchJSON',  params=payload)
            if (r.status_code != 200 or r.text == '{"postalCodes":[]}'):
                bundesland.append('BY')
                print(r.url)
                print('Bundesland = BY')
                continue
            lol = r.json()
        except requests.exceptions.ChunkedEncodingError:
            print('retrying...')
            continue
        print(r.url)
        print(lol['postalCodes'][0]['ISO3166-2'])
        #lol = r.json()
        bundesland.append(lol['postalCodes'][0]['ISO3166-2'])
    else:
        bundesland.append('BY')

In [ ]:
df['land'] = bundesland

In [ ]:
df.set_index('bhf').loc['Leipzig Halle Flughafen']

In [ ]:
df.to_csv('wetterdaten/bahnhof_to_weather_location.csv', index=False)

In [ ]:
import pandas as pd
from datetime import datetime
import json
import sys

frame = pd.read_csv('../data/combinedData/alltrains2019.csv',encoding='UTF-8',compression='zip', engine='c')
bhfs = pd.read_csv('../stations.csv',encoding='UTF-8')
bhfs = bhfs[bhfs.loc[:,'filename'].notnull()].set_index("filename")
frame = frame.set_index("bhf")

filename = 'Halle.json'
with open('../data/wetterdaten/wetterdaten2019/' + filename) as json_file:
    weather_json = pd.DataFrame.from_dict(json.load(json_file)['history_1h'])
try:
    bhf = frame.loc[frame.index.isin(bhfs.loc[filename.replace(".json", ""), "name"].tolist())].reset_index()
except:
    print(str(sys.exc_info()[0]) + " Error " + bhfs.loc[filename.replace(".json", ""), "name"])
    bhf = frame.loc[frame2.index.isin([bhfs.loc[filename.replace(".json", ""), "name"]])].reset_index()

bhf = bhf.reset_index()
data = pd.DataFrame()
if len(bhf) < 1:
    exit()
#weather_json.set_index("time", inplace = True)
for i, x in bhf.iterrows():
    date = datetime.strptime(bhf.at[i,'date'], '%Y-%m-%d')
    ####-Add-weather-to-trainData-####

    # 1. nächste Uhrzeit finden
    arr_time = bhf.at[i, 'arr'] if bhf.at[i, 'arr'] != '99:99' else bhf.at[i, 'dep']
    try: #sometimes there are times like '14:99'. We are skipping those.
        arr_time = pd.to_datetime(date.strftime('%Y-%m-%d') + 'T' + arr_time)
    except:
        print("except")
        continue
    x = weather_json['time'].astype('datetime64[h]').reset_index(drop=True)



    #Calculate the timedelta amount
    v = x.apply(lambda dep_time: dep_time - arr_time if dep_time > arr_time else arr_time - dep_time)


    #find nearest time
    date = x[v.idxmin()].strftime('%Y-%m-%d %H:%M')
    #print(date)
    #all data with matching date and time
    data =  weather_json.loc[weather_json['time'].isin([date]), :] #.astype('datetime64[ns]')
    data = data.reset_index(drop=True)
    # 2. daten eintragen
    bhf.at[i, 'sealevelpressure'] = data.at[0, 'sealevelpressure']
    bhf.at[i, 'temperature'] = data.at[0, 'temperature']
    bhf.at[i, 'precipitation'] = data.at[0, 'precipitation']
    bhf.at[i, 'snowfraction'] = data.at[0, 'snowfraction']
    bhf.at[i, 'winddirection'] = data.at[0, 'winddirection']
    bhf.at[i, 'windspeed'] = data.at[0, 'windspeed']
    bhf.at[i, 'relativehumidity'] = data.at[0, 'relativehumidity']
    ####--------------------------####
    if ((i % 100) == 0):
        print('|', end='')
bhf.to_csv('../data/wetterdaten/combined/' + filename.replace(".json", "") + '.csv',encoding='UTF-8')

In [ ]:
import os

In [ ]:
bhfs = pd.read_csv('../stations.csv',encoding='UTF-8')
bhfs2 = bhfs.set_index("city_search_string")
path = '../data/wetterdaten/wetterdaten2019/'

for filename in os.listdir(path):
    if not filename.replace(".json", "")  in bhfs2['filename'].tolist():
        os.system('python3 ../add_data.py ' + filename +' >& /dev/null &')